In [1]:
import pandas as pd
import numpy as np

In [20]:
df_orgs = pd.read_csv('../joined-data/simplified_matches.csv')

df_uw = pd.read_excel('../cra-data/cre-uw-checking 2.xlsx', sheet_name="UW Orgs")
df_uw = df_uw.rename(columns={'CHARITABLE REGISTRATION NUMBER (BN)': 'CRA_BN_ID'})
df_uw = df_uw[df_uw['CRA_BN_ID'].notna()]

In [25]:
df_uw

,Funded Agency - CRA Name,CRA_BN_ID,Category,Sub Category,City,Reason,Region
2,Council of Agencies Serving South Asians,135875599RR0001,NaN,NaN,NaN,BN Not in CRA (in 2021),NaN
7,Human Endeavour Incorporation,857332704RR0001,NaN,NaN,NaN,BN Not in CRA (in 2021),NaN
8,Refugee Career Jumpstart Project,787710490RR0001,NaN,NaN,NaN,BN Not in CRA (in 2021),NaN
9,Brampton 9235 Outreach Corp,132503731RR0001,NaN,NaN,NaN,BN Not in CRA (in 2021),NaN
19,The Reading Partnership,727408932RR0001,NaN,NaN,NaN,BN Not in CRA (in 2021),NaN
...,...,...,...,...,...,...,...
273,Restoration and Empowerment For Social Transit...,811570365RR0001,160.0,19.0,BRAMPTON,Not joined to 211 data,Peel
274,The Canadian Red Cross Society La Soci�t� Cana...,119219814RR0001,100.0,99.0,OTTAWA,Postal Code not included in filter,NaN
275,Dnaagdawenmag Binnoojiiyag Child and Family Se...,830618088RR0001,160.0,19.0,HIAWATHA,Postal Code not included in filter,NaN
276,Routes Connecting Communities Inc.,888551868RR0001,1.0,99.0,SUTTON WEST,Postal Code not included in filter,York


In [24]:
len(set(df_uw['CRA_BN_ID']).intersection(set(df_orgs['CRA_BN_ID'])))

183

In [21]:
df_uw

,Funded Agency - CRA Name,CRA_BN_ID,Category,Sub Category,City,Reason,Region
2,Council of Agencies Serving South Asians,135875599RR0001,NaN,NaN,NaN,BN Not in CRA (in 2021),NaN
7,Human Endeavour Incorporation,857332704RR0001,NaN,NaN,NaN,BN Not in CRA (in 2021),NaN
8,Refugee Career Jumpstart Project,787710490RR0001,NaN,NaN,NaN,BN Not in CRA (in 2021),NaN
9,Brampton 9235 Outreach Corp,132503731RR0001,NaN,NaN,NaN,BN Not in CRA (in 2021),NaN
19,The Reading Partnership,727408932RR0001,NaN,NaN,NaN,BN Not in CRA (in 2021),NaN
...,...,...,...,...,...,...,...
273,Restoration and Empowerment For Social Transit...,811570365RR0001,160.0,19.0,BRAMPTON,Not joined to 211 data,Peel
274,The Canadian Red Cross Society La Soci�t� Cana...,119219814RR0001,100.0,99.0,OTTAWA,Postal Code not included in filter,NaN
275,Dnaagdawenmag Binnoojiiyag Child and Family Se...,830618088RR0001,160.0,19.0,HIAWATHA,Postal Code not included in filter,NaN
276,Routes Connecting Communities Inc.,888551868RR0001,1.0,99.0,SUTTON WEST,Postal Code not included in filter,York


In [22]:
pd.merge(
    left = df_uw,
    right = df_orgs,
    on = 'CRA_BN_ID',
    how = 'left'
)

,Funded Agency - CRA Name,CRA_BN_ID,Category,Sub Category,City,Reason,Region,CRA_Organization_Name,211_Organization_Name,211_Location_Name,211_Address,211_Postal_Code,X_Coordinate,Y_Coordinate,Match_Method
0,Council of Agencies Serving South Asians,135875599RR0001,NaN,NaN,NaN,BN Not in CRA (in 2021),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Human Endeavour Incorporation,857332704RR0001,NaN,NaN,NaN,BN Not in CRA (in 2021),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Refugee Career Jumpstart Project,787710490RR0001,NaN,NaN,NaN,BN Not in CRA (in 2021),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Brampton 9235 Outreach Corp,132503731RR0001,NaN,NaN,NaN,BN Not in CRA (in 2021),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Reading Partnership,727408932RR0001,NaN,NaN,NaN,BN Not in CRA (in 2021),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,Restoration and Empowerment For Social Transit...,811570365RR0001,160.0,19.0,BRAMPTON,Not joined to 211 data,Peel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
809,The Canadian Red Cross Society La Soci�t� Cana...,119219814RR0001,100.0,99.0,OTTAWA,Postal Code not included in filter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
810,Dnaagdawenmag Binnoojiiyag Child and Family Se...,830618088RR0001,160.0,19.0,HIAWATHA,Postal Code not included in filter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
811,Routes Connecting Communities Inc.,888551868RR0001,1.0,99.0,SUTTON WEST,Postal Code not included in filter,York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
